In [3]:
import requests
import tempfile
import tarfile
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

temp_dir = tempfile.mkdtemp()
data_dir = os.path.join(temp_dir, 'Task_04_Hippocampus')

if not os.path.exists(data_dir):
    tar_path = os.path.join(temp_dir, 'data.tar')
    if not os.path.exists(tar_path):
        print('Downloading Data')
        download_file_from_google_drive('1RzPB1_bqzQhlWvU-YGvZzhx2omcDh38C', 
                                        os.path.join(temp_dir, 'test.tar'))
    print('Extracting Data')
    tarfile.TarFile(os.path.join(temp_dir, 'test.tar')).extractall(temp_dir)
    print('Success!')
data_dir = os.path.join(temp_dir, 'Task04_Hippocampus')

Extracting Data
Success!


In [12]:
print(temp_dir)
data_dir = os.path.join(temp_dir, 'Task04_Hippocampus')

/var/folders/3h/nw_brd4n0hs9y_wv_frs8kww0000gn/T/tmpeu8f2wxn


In [14]:
import json
with open(os.path.join(data_dir, 'dataset.json')) as f:
    content = json.load(f)

print(content)
print(type(content))

sTr/hippocampus_234.nii.gz'}, {'image': './imagesTr/hippocampus_257.nii.gz', 'label': './labelsTr/hippocampus_257.nii.gz'}, {'image': './imagesTr/hippocampus_302.nii.gz', 'label': './labelsTr/hippocampus_302.nii.gz'}, {'image': './imagesTr/hippocampus_361.nii.gz', 'label': './labelsTr/hippocampus_361.nii.gz'}, {'image': './imagesTr/hippocampus_261.nii.gz', 'label': './labelsTr/hippocampus_261.nii.gz'}, {'image': './imagesTr/hippocampus_210.nii.gz', 'label': './labelsTr/hippocampus_210.nii.gz'}, {'image': './imagesTr/hippocampus_310.nii.gz', 'label': './labelsTr/hippocampus_310.nii.gz'}, {'image': './imagesTr/hippocampus_373.nii.gz', 'label': './labelsTr/hippocampus_373.nii.gz'}, {'image': './imagesTr/hippocampus_355.nii.gz', 'label': './labelsTr/hippocampus_355.nii.gz'}, {'image': './imagesTr/hippocampus_328.nii.gz', 'label': './labelsTr/hippocampus_328.nii.gz'}, {'image': './imagesTr/hippocampus_228.nii.gz', 'label': './labelsTr/hippocampus_228.nii.gz'}, {'image': './imagesTr/hippocam

In [18]:
list(content.keys())
print(type(content['training']))
print(content['training'][0])
#print(content['training'])

<class 'list'>
{'image': './imagesTr/hippocampus_367.nii.gz', 'label': './labelsTr/hippocampus_367.nii.gz'}


In [28]:
import SimpleITK as sitk
import json
from rising import loading
from rising.loading import Dataset
import torch
class NiiDataset(Dataset):
    def __init__(self, train: bool, data_dir: str):
        with open(os.path.join(data_dir, 'dataset.json')) as f:
            content = json.load(f)
            if train:
                data = content['training']
            else:
                data = content['test']
            
            self.data = data
            self.data_dir = data_dir

    def __getitem__(self, item: int):
        sample = self.data[item]
        img = sitk.GetArrayFromImage(
            sitk.ReadImage(os.path.join(self.data_dir, sample['image'])))

        # add channel dim if necesary
        if img.ndim == 3:
            img = img[None]

        label = sitk.GetArrayFromImage(
            sitk.ReadImage(os.path.join(self.data_dir, sample['label'])))
        
        # remove channel dim if necessary
        if label.ndim == 4:
            label = label[0]
        return {'data': torch.from_numpy(img), 
                'label': torch.from_numpy(label)}

    def __len__(self):
        return len(self.data)

dataset = NiiDataset(True, data_dir)

In [33]:
from tqdm import tqdm
sizes_data, sizes_label = [], []
for idx in tqdm(range(len(dataset))):
    sample = dataset[idx]
    sizes_data.append(torch.tensor(sample['data'].shape))
    sizes_label.append(torch.tensor(sample['label'].shape))

torch.stack(sizes_data).float().mean(0)

# Resize with per sample trafo
# apply augmentation per batch
# Simple Unet Lightning
# CrossEntropy + SoftDice


100%|██████████| 260/260 [00:01<00:00, 252.53it/s]


tensor([ 1.0000, 35.6538, 49.9808, 35.3769])